In [1]:
import logging
import time

# Configure logger
logger: logging.Logger = logging.getLogger("run_workflow_logger")
logger.setLevel(logging.INFO)
logger.propagate = False  # Prevent duplicate logs from parent loggers

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [2]:
start_time = time.time()  

logger.info("Notebook execution started.")

2025-07-03 22:41:00 - INFO - Notebook execution started.


In [3]:
%%time

%pip install -r ../requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.
CPU times: user 21.6 ms, sys: 15.6 ms, total: 37.2 ms
Wall time: 1.29 s


In [ ]:
import os
import sys
import logging

# Define the relative path to the 'src' directory (two levels up from current working directory)
src_path = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add 'src' directory to system path for module imports (e.g., utils)
if src_path not in sys.path:
    sys.path.append(src_path)

# === Standard Library Imports ===
import os
import sys
import logging
import json
import time
import warnings
from datetime import datetime
from pathlib import Path

# === Third-Party Imports ===
import numpy as np
import pandas as pd
import evaluate
import webvtt
import mlflow
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from operator import itemgetter

# === Project-Specific Imports (from src.utils) ===
from src.utils import (
    load_config_and_secrets,
    configure_proxy,
    initialize_llm,
    configure_hf_cache
)
from src.prompt_templates import format_chunk_summarization_prompt

## Model Service Registration

In this example, we illustrate a different approach to create a text summarizer. Instead of splitting the text into topics and summarize the topics individually, this model service provides a REST API endpoint to allow summarization of an entire text, in a single call to the model.

## Text Summarization Service

This section demonstrates how to use our TextSummarizationService from the src/service directory. This approach improves code organization by separating the service implementation from the notebook, making it easier to maintain and update.

In [5]:
%%time

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# In case you just want to run this cell without the rest of the notebook 
# (you still need to install the requirements and run the import block), run the following block:
CONFIG_PATH = "../configs/config.yaml"
SECRETS_PATH = "../configs/secrets.yaml"
MODEL_PATH = "/home/jovyan/datafabric/meta-llama3.1-8b-Q8/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"
config, secrets = load_config_and_secrets(CONFIG_PATH, SECRETS_PATH)

# Import the TextSummarizationService class
from core.service.text_summarization_service import TextSummarizationService

mlflow.set_tracking_uri('/phoenix/mlflow')
# Set up the MLflow experiment
mlflow.set_experiment("Summarization_Service")

# Define path to the model

# Check if the model file exists
if not os.path.exists(MODEL_PATH):
    print(f"Warning: Model file not found at {MODEL_PATH}. You may need to update the path.")

# Define demo folder path
DEMO_FOLDER = "../demo"

#Only logs the model path in the case where it is local
if config["model_source"] == "local":
    model_path = MODEL_PATH
else:
    model_path = None


# Use the TextSummarizationService's log_model method to register the model in MLflow
with mlflow.start_run(run_name="Text_Summarization_Service") as run:
    # Log and register the model using the service's classmethod
    TextSummarizationService.log_model(
        artifact_path="text_summarization_service",
        secrets_path=SECRETS_PATH,
        config_path=CONFIG_PATH,
        model_path=model_path,
        demo_folder=DEMO_FOLDER
    )
    
    # Register the model in MLflow Model Registry
    model_uri = f"runs:/{run.info.run_id}/text_summarization_service"
    mlflow.register_model(model_uri=model_uri, name="Text_Summarization_Service")
    print(f"Model registered successfully with run ID: {run.info.run_id}")

/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
2025/07/03 22:41:11 INFO mlflow.tracking.fluent: Experiment with name 'Summarization_Service' does not exist. Creating a new experiment.
/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/__init__.py:3212: UserWarning: An input example was not provided when logging the model. To ensure the model signature 

2025-07-03 22:45:45,538 - INFO - Model and artifacts successfully registered in MLflow.
Successfully registered model 'Text_Summarization_Service'.


Model registered successfully with run ID: d3d1689ca902408a9c5af5a566d7fb02
CPU times: user 1.16 s, sys: 21.1 s, total: 22.3 s
Wall time: 4min 35s


Created version '1' of model 'Text_Summarization_Service'.


In [6]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")

2025-07-03 22:45:46 - INFO - ⏱️ Total execution time: 4m 45.90s
2025-07-03 22:45:46 - INFO - ✅ Notebook execution completed successfully.


Built with ❤️ using [**HP AI Studio**](https://hp.com/ai-studio).